In [3]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
import os

In [4]:
window=tk.Tk()
window.title("Face recogonition system")
l1=tk.Label(window,text="Name",font=("Algerian",20))
l1.grid(column=0,row=0)
t1=tk.Entry(window,width=50,bd=5)
t1.grid(column=1,row=0)
l2=tk.Label(window,text="Age",font=("Algerian",20))
l2.grid(column=0,row=1)
t2=tk.Entry(window,width=50,bd=5)
t2.grid(column=1,row=1)
def distance_camera():
    
    # variables 
    # distance from camera to object(face) measured
    Known_distance =45 #centimeter
    # width of face in the real world or Object Plane
    Known_width =12 
    # Colors
    GREEN = (0,255,0)
    RED = (0,0,255)
    WHITE = (255,255,255)
    fonts = cv2.FONT_HERSHEY_COMPLEX
    cap = cv2.VideoCapture(0)
    # face detector object
    face_detector = cv2.CascadeClassifier("C:/Users/Akhilesh/Haarcascades/haarcascade_frontalface_default.xml")
    # focal length finder function  
    def FocalLength(measured_distance, real_width, width_in_rf_image):
        focal_length = (width_in_rf_image* measured_distance)/ real_width
        return focal_length
    # distance estimation function
    def Distance_finder (Focal_Length, real_face_width, face_width_in_frame):
        distance = (real_face_width * Focal_Length)/face_width_in_frame
        return distance

    def face_data(image):
        face_width = 0 
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(image, 1.3, 5)
        for (x, y, h, w) in faces:
            cv2.rectangle(image, (x, y), (x+w, y+h), WHITE, 1)
            face_width = w

        return face_width

    # reading reference image from directory
    ref_image = cv2.imread("C:/Users/Akhilesh/opencv_frame_1.jpg")

    ref_image_face_width = face_data(ref_image)
    Focal_length_found = FocalLength(Known_distance, Known_width, ref_image_face_width)
    print(Focal_length_found)
    cv2.imshow("ref_image", ref_image)

    while True:
        _, frame = cap.read()

        # calling face_data function
        face_width_in_frame = face_data(frame)
        # finding the distance by calling function Distance finder
        if face_width_in_frame !=0:
            Distance = Distance_finder(Focal_length_found, Known_width,face_width_in_frame)
        # Drwaing Text on the screen
            cv2.putText(frame, f"Distance = {Distance}", (50,50), fonts,1, (WHITE),2)
        cv2.imshow("frame", frame )
        if cv2.waitKey(1)==ord("q"):
            break 
    cap.release()
    cv2.destroyAllWindows()
b2=tk.Button(window,text="Distance_camera",font=("Algerian",20),bg="yellow",fg="red",command=distance_camera)
b2.grid(column=1,row=6)
def generate_dataset():
    face_classifier = cv2.CascadeClassifier('C:/Users/Akhilesh/Haarcascades/haarcascade_frontalface_default.xml')
    def face_extractor(img):
        # Function detects faces and returns the cropped face
        # If no face detected, it returns the input image
    
        #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(img, 1.3, 5)
    
        if faces is ():
            return None
    
        # Crop all faces found
        for (x,y,w,h) in faces:
            x=x-10
            y=y-10
            cropped_face = img[y:y+h+50, x:x+w+50]

        return cropped_face

    # Initialize Webcam
    cap = cv2.VideoCapture(0)
    count = 0

    # Collect 1 samples of your face from webcam input
    while True:
        ret, frame = cap.read()
        if face_extractor(frame) is not None:
            count += 1
            face = cv2.resize(face_extractor(frame), (400, 400))
            #face = cv2.cvtColor(face, cv2.COLOR_GRAY2BGR)

            # Save file in specified directory with unique name
            file_name_path = "opencv_frame_{}.jpg".format(count)
            cv2.imwrite(file_name_path, face)

            # Put count on images and display live count
            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Cropper', face)
        
        else:
            print("Face not found")
            pass

        if cv2.waitKey(1) == 13 or count == 1: #13 is the Enter Key
            break
        
    cap.release()
    cv2.destroyAllWindows()      
    
b3=tk.Button(window,text=" generate dataset",font=("Algerian",20),bg="yellow",fg="red",command=generate_dataset)
b3.grid(column=2,row=6)
def opencv_tracker():
    #tracker = cv2.TrackerBoosting_create()
    #tracker = cv2.TrackerMIL_create()
    #tracker = cv2.TrackerKCF_create()
    #tracker = cv2.TrackerTLD_create()
    #tracker = cv2.TrackerMedianFlow_create()
    #tracker = cv2.TrackerCSRT_create()
    tracker = cv2.TrackerMOSSE_create()
    cap = cv2.VideoCapture(0)
    # TRACKER INITIALIZATION
    success, frame = cap.read()
    bbox = cv2.selectROI("Tracking",frame, False)
    tracker.init(frame, bbox)


    def drawBox(img,bbox):
        x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
        cv2.rectangle(img, (x, y), ((x + w), (y + h)), (255, 0, 255), 3, 3 )
        cv2.putText(img, "Tracking", (100, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


    while True:

        timer = cv2.getTickCount()
        success, img = cap.read()
        success, bbox = tracker.update(img)

        if success:
            drawBox(img,bbox)
        else:
            cv2.putText(img, "Lost", (100, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.rectangle(img,(15,15),(200,90),(255,0,255),2)
        cv2.putText(img, "Fps:", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,255), 2);
        cv2.putText(img, "Status:", (20, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2);


        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
        if fps>60: myColor = (20,230,20)
        elif fps>20: myColor = (230,20,20)
        else: myColor = (20,20,230)
        cv2.putText(img,str(int(fps)), (75, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, myColor, 2);

        cv2.imshow("Tracking", img)
        if cv2.waitKey(1) & 0xff == ord('q'):
               break

b4=tk.Button(window,text="opencv_tracker",font=("Algerian",20),bg="yellow",fg="red",command=opencv_tracker)
b4.grid(column=4,row=6)


window.geometry("800x200")
window.mainloop()

<>:79: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:79: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-4-6fe3d04858c4>:79: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
997.5
